# Hands-on: Training and deploying Question Answering with BERT

Pre-trained language representations have been shown to improve many downstream NLP tasks such as question answering, and natural language inference. Devlin, Jacob, et al proposed BERT [1] (Bidirectional Encoder Representations from Transformers), which fine-tunes deep bidirectional representations on a wide range of tasks with minimal task-specific parameters, and obtained state- of-the-art results.

In this tutorial, we will focus on adapting the BERT model for the question answering task on the SQuAD dataset. Specifically, we will:

- understand how to pre-process the SQuAD dataset to leverage the learnt representation in BERT,
- adapt the BERT model to the question answering task, and
- load a trained model to perform inference on the SQuAD dataset

## Sagemaker configuration

This notebook requires mxnet-cu101 >= 1.6.0b20191102, gluonnlp >= 0.8.1
We can create a sagemaker notebook instance with the lifecycle configuration file: sagemaker-lifecycle.config

In [ ]:
# One time script
# !bash sagemaker-lifecycle.config && pip install mxnet-cu101 --pre -U

In [1]:
!pip list | grep mxnet
!pip list | grep gluonnlp

keras-mxnet                        2.2.4.2       
mxnet-cu101                        1.6.0b20191122
mxnet-model-server                 1.0.5         
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
gluonnlp                           0.9.0.dev0    
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load MXNet and GluonNLP

We first import the libraries:

In [2]:
import argparse, collections, time, logging
import json
import os
import io
import copy
import random
import warnings

import numpy as np
import gluonnlp as nlp
import mxnet as mx
import bert
import qa_utils

from gluonnlp.data import SQuAD
from bert.model.qa import BertForQALoss, BertForQA
from bert.data.qa import SQuADTransform, preprocess_dataset
from bert.bert_qa_evaluate import get_F1_EM, predict, PredResult

# Hyperparameters
parser = argparse.ArgumentParser('BERT finetuning')
parser.add_argument('--epochs', type=int, default=3)
parser.add_argument('--batch_size', default=32)
parser.add_argument('--num_epochs', default=1)
parser.add_argument('--lr', default=5e-5)


args = parser.parse_args([])


epochs = args.epochs
batch_size = args.batch_size
num_epochs = args.num_epochs
lr = args.lr

# output_dir = args.output_dir
# if not os.path.exists(output_dir):
#     os.mkdir(output_dir)
# test_batch_size = args.test_batch_size
# optimizer = args.optimizer
# accumulate = args.accumulate
# warmup_ratio = args.warmup_ratio
# log_interval = args.log_interval
# max_seq_length = args.max_seq_length
# doc_stride = args.doc_stride
# max_query_length = args.max_query_length
# n_best_size = args.n_best_size

## Inspect the SQuAD Dataset

Then we take a look at the Stanford Question Answering Dataset (SQuAD). The dataset can be downloaded using the `nlp.data.SQuAD` API. In this tutorial, we create a small dataset with 3 samples from the SQuAD dataset for demonstration purpose.

The question answering task on the SQuAD dataset is setup the following way. For each sample in the dataset, a context is provided. The context is usually a long paragraph which contains lots of information. Then a question asked based on the context. The goal is to find the text span in the context that answers the question in the sample.

In [3]:
full_data = nlp.data.SQuAD(segment='dev', version='1.1')
# loading a subset of the dev set of SQuAD
num_target_samples = 3
target_samples = [full_data[i] for i in range(num_target_samples)]
dataset = mx.gluon.data.SimpleDataset(target_samples)
print('Number of samples in the created dataset subsampled from SQuAD = %d'%len(dataset))

Number of samples in the created dataset subsampled from SQuAD = 3


Let's take a look at a sample from the dataset. In this sample, the question is about the location of the game, with a description about the Super Bowl 50 game as the context. Note that three different answer spans are correct for this question, and they start from index 403, 355 and 355 in the context respectively.

In [4]:
sample = dataset[2]

context_idx = 3

print('\nContext:\n')
print(sample[context_idx])


Context:

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [5]:
question_idx = 2
answer_idx = 4
answer_pos_idx = 5

print("\nQuestion")
print(sample[question_idx])
print("\nCorrect Answer Spans")
print(sample[answer_idx])
print("\nAnswer Span Start Indices:")
print(sample[answer_pos_idx])


Question
Where did Super Bowl 50 take place?

Correct Answer Spans
['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]

Answer Span Start Indices:
[403, 355, 355]


## Data Pre-processing for QA with BERT

Recall that during BERT pre-training, it takes a sentence pair as the input, separated by the 'SEP' special token. For SQuAD, we can feed the context-question pair as the sentence pair input. To use BERT to predict the starting and ending span of the answer, we can add a classification layer for each token in the context texts, to predict if a token is the start or the end of the answer span. 

![qa](natural_language_understanding/qa.png)

In the next few code blocks, we will work on pre-processing the samples in the SQuAD dataset in the desired format with these special separators. 


### Get Pre-trained BERT Model

First, let's use the *get_model* API in GluonNLP to get the model definition for BERT, and the vocabulary used for the BERT model. Note that we discard the pooler and classifier layers used for the next sentence prediction task, as well as the decoder layers for the masked language model task during the BERT pre-training phase. These layers are not useful for predicting the starting and ending indices of the answer span.

The list of pre-trained BERT models available in GluonNLP can be found [here](http://gluon-nlp.mxnet.io/model_zoo/bert/index.html).

In [6]:
bert_model, vocab = nlp.model.get_model('bert_12_768_12',
                                        dataset_name='book_corpus_wiki_en_uncased',
                                        use_classifier=False,
                                        use_decoder=False,
                                        use_pooler=False,
                                        pretrained=False)

Vocab file is not found. Downloading.


Note that there are several special tokens in the vocabulary for BERT. In particular, the `[SEP]` token is used for separating the sentence pairs, and the `[CLS]` token is added at the beginning of the sentence pairs. They will be used to pre-process the SQuAD dataset later.

In [7]:
print(vocab)

Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")


### Tokenization

The second step is to process the samples using the same tokenizer used for BERT, which is provided as the `BERTTokenizer` API in GluonNLP. Note that instead of word level and character level representation, BERT uses subwords to represent a word, separated `##`. 

In the following example, the word `suspending` is tokenized as two subwords (`suspend` and `##ing`), and `numerals` is tokenized as three subwords (`nu`, `##meral`, `##s`).

In [8]:
tokenizer = nlp.data.BERTTokenizer(vocab=vocab, lower=True)

tokenizer("as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals")

['as',
 'well',
 'as',
 'temporarily',
 'suspend',
 '##ing',
 'the',
 'tradition',
 'of',
 'naming',
 'each',
 'super',
 'bowl',
 'game',
 'with',
 'roman',
 'nu',
 '##meral',
 '##s']

### Sentence Pair Composition

With the tokenizer inplace, we are ready to process the question-context texts and compose sentence pairs. The functionality is available via the `SQuADTransform` API. 

In [9]:
transform = bert.data.qa.SQuADTransform(tokenizer, is_pad=False, is_training=False, do_lookup=False)
dev_data_transform, _ = bert.data.qa.preprocess_dataset(dataset, transform)
logging.info('The number of examples after preprocessing:{}'.format(len(dev_data_transform)))

Done! Transform dataset costs 0.19 seconds.


Let's take a look at the sample after the transformation:

In [10]:
sample = dev_data_transform[2]
print('\nsegment type: \n' + str(sample[2]))
print('\ntext length: ' + str(sample[3]))
print('\nsentence pair: \n' + str(sample[1]))


segment type: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

text length: 168

sentence pair: 
['[CLS]', 'where', 'did', 'super', 'bowl', '50', 'take', 'place', '?', '[SEP]', 'super', 'bowl', '50', 'was', 'an', 'american', 'football', 'game', 'to', 'determine', 'the', 'champion', 'of', 'the', 'national', 'football', 'league', '(', 'nfl', ')', 'for', 'the', '2015', 'season', '.', 'the', 'american', 'football', 'conference', '(', 'afc', ')', 'champion', 'denver', 'broncos', 'defeated', 'the', 'national', 'football', 'conference', '(', 

### Vocabulary Lookup

Finally, we convert the transformed texts to subword indices, which are used to contructor NDArrays as the inputs to the model.

In [11]:
def vocab_lookup(example_id, subwords, type_ids, length, start, end):
    indices = vocab[subwords]
    return example_id, indices, type_ids, length, start, end

dev_data_transform = dev_data_transform.transform(vocab_lookup, lazy=False)
print(dev_data_transform[2][1])

[2, 2073, 2106, 3565, 4605, 2753, 2202, 2173, 1029, 3, 3565, 4605, 2753, 2001, 2019, 2137, 2374, 2208, 2000, 5646, 1996, 3410, 1997, 1996, 2120, 2374, 2223, 1006, 5088, 1007, 2005, 1996, 2325, 2161, 1012, 1996, 2137, 2374, 3034, 1006, 10511, 1007, 3410, 7573, 14169, 3249, 1996, 2120, 2374, 3034, 1006, 22309, 1007, 3410, 3792, 12915, 2484, 1516, 2184, 2000, 7796, 2037, 2353, 3565, 4605, 2516, 1012, 1996, 2208, 2001, 2209, 2006, 2337, 1021, 1010, 2355, 1010, 2012, 11902, 1005, 1055, 3346, 1999, 1996, 2624, 3799, 3016, 2181, 2012, 4203, 10254, 1010, 2662, 1012, 2004, 2023, 2001, 1996, 12951, 3565, 4605, 1010, 1996, 2223, 13155, 1996, 1000, 3585, 5315, 1000, 2007, 2536, 2751, 1011, 11773, 11107, 1010, 2004, 2092, 2004, 8184, 28324, 2075, 1996, 4535, 1997, 10324, 2169, 3565, 4605, 2208, 2007, 3142, 16371, 28990, 2015, 1006, 2104, 2029, 1996, 2208, 2052, 2031, 2042, 2124, 2004, 1000, 3565, 4605, 1048, 1000, 1007, 1010, 2061, 2008, 1996, 8154, 2071, 14500, 3444, 1996, 5640, 16371, 28990, 2015

## Model Definition

After the data is processed, we can define the model that uses the representation produced by BERT for predicting the starting and ending positions of the answer span.

We download a BERT model trained on the SQuAD dataset, prepare the dataloader.

In [12]:
net = BertForQA(bert_model)
ctx = mx.gpu(0)
ckpt = qa_utils.download_qa_ckpt()
net.load_parameters(ckpt, ctx=ctx)

batch_size = 1
dev_dataloader = mx.gluon.data.DataLoader(
    dev_data_transform, batch_size=batch_size, shuffle=False)

Downloaded checkpoint to ./temp/bert_qa-7eb11865.params


In [14]:
all_results = collections.defaultdict(list)

total_num = 0
for data in dev_dataloader:
    example_ids, inputs, token_types, valid_length, _, _ = data
    total_num += len(inputs)
    batch_size = inputs.shape[0]
    output = net(inputs.astype('float32').as_in_context(ctx),
                               token_types.astype('float32').as_in_context(ctx),
                               valid_length.astype('float32').as_in_context(ctx))
    pred_start, pred_end = mx.nd.split(output, axis=2, num_outputs=2)
    example_ids = example_ids.asnumpy().tolist()
    pred_start = pred_start.reshape(batch_size, -1).asnumpy()
    pred_end = pred_end.reshape(batch_size, -1).asnumpy()
    
    for example_id, start, end in zip(example_ids, pred_start, pred_end):
        all_results[example_id].append(PredResult(start=start, end=end))

In [15]:
qa_utils.predict(dataset, all_results, vocab)


Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.

Question: which nfl team represented the afc at super bowl 50 ?

Top predictions: 
99.36% 	 Denver Broncos
0.23% 	 The American Football Conference (AFC) champion Denver Broncos
0.20% 	 Broncos


Context: Super Bo

### Let's Train the Model

Now we can put all the pieces together, and start fine-tuning the model with a few epochs.

In [ ]:
# net = BertForQA(bert=bert_model)
# nlp.utils.load_parameters(net, pretrained_bert_parameters, ctx=ctx,
#                           ignore_extra=True, cast_dtype=True)
net.span_classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
net.hybridize(static_alloc=True)

loss_function = BertForQALoss()
loss_function.hybridize(static_alloc=True)

In [ ]:

batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Stack(),
    nlp.data.batchify.Pad(axis=0, pad_val=vocab[vocab.padding_token]),
    nlp.data.batchify.Pad(axis=0, pad_val=vocab[vocab.padding_token]),
    nlp.data.batchify.Stack('float32'),
    nlp.data.batchify.Stack('float32'),
    nlp.data.batchify.Stack('float32'))

np.random.seed(6)
random.seed(6)
mx.random.seed(6)

log = logging.getLogger('gluonnlp')
log.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    fmt='%(levelname)s:%(name)s:%(asctime)s %(message)s', datefmt='%H:%M:%S')


segment = 'train'  # if not args.debug else 'dev'
log.info('Loading %s data...', segment)
#     if version_2:
#         train_data = SQuAD(segment, version='2.0')
#     else:
train_data = SQuAD(segment, version='1.1')
#     if args.debug:
#         sampled_data = [train_data[i] for i in range(1000)]
#         train_data = mx.gluon.data.SimpleDataset(sampled_data)
log.info('Number of records in Train data:{}'.format(len(train_data)))

train_data_transform, _ = preprocess_dataset(
    train_data, SQuADTransform(
        copy.copy(tokenizer),
        max_seq_length=max_seq_length,
        doc_stride=doc_stride,
        max_query_length=max_query_length,
        is_pad=True,
        is_training=True))
log.info('The number of examples after preprocessing:{}'.format(
    len(train_data_transform)))

train_dataloader = mx.gluon.data.DataLoader(
    train_data_transform, batchify_fn=batchify_fn,
    batch_size=batch_size, num_workers=4, shuffle=True)

In [ ]:

def train(log, train_data_transform, train_dataloader):
    """Training function."""

    log.info('Start Training')

    optimizer_params = {'learning_rate': lr}
    trainer = mx.gluon.Trainer(net.collect_params(), optimizer,
                               optimizer_params, update_on_kvstore=False)

    num_train_examples = len(train_data_transform)
    step_size = batch_size * accumulate if accumulate else batch_size
    num_train_steps = int(num_train_examples / step_size * epochs)
    num_warmup_steps = int(num_train_steps * warmup_ratio)
    step_num = 0
    
    def set_new_lr(step_num, batch_id):
        """set new learning rate"""
        # set grad to zero for gradient accumulation
        if accumulate:
            if batch_id % accumulate == 0:
                net.collect_params().zero_grad()
                step_num += 1
        else:
            step_num += 1
        # learning rate schedule
        # Notice that this learning rate scheduler is adapted from traditional linear learning
        # rate scheduler where step_num >= num_warmup_steps, new_lr = 1 - step_num/num_train_steps
        if step_num < num_warmup_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            offset = (step_num - num_warmup_steps) * lr / \
                (num_train_steps - num_warmup_steps)
            new_lr = lr - offset
        trainer.set_learning_rate(new_lr)
        return step_num

    # Do not apply weight decay on LayerNorm and bias terms
    for _, v in net.collect_params('.*beta|.*gamma|.*bias').items():
        v.wd_mult = 0.0
    # Collect differentiable parameters
    params = [p for p in net.collect_params().values()
              if p.grad_req != 'null']
    # Set grad_req if gradient accumulation is required
    if accumulate:
        for p in params:
            p.grad_req = 'add'

    epoch_tic = time.time()
    total_num = 0
    log_num = 0
    for epoch_id in range(epochs):
        step_loss = 0.0
        tic = time.time()
        for batch_id, data in enumerate(train_dataloader):
            # set new lr
            step_num = set_new_lr(step_num, batch_id)
            # forward and backward
            with mx.autograd.record():
                _, inputs, token_types, valid_length, start_label, end_label = data

                log_num += len(inputs)
                total_num += len(inputs)

                out = net(inputs.astype('float32').as_in_context(ctx),
                          token_types.astype('float32').as_in_context(ctx),
                          valid_length.astype('float32').as_in_context(ctx))

                ls = loss_function(out, [
                    start_label.astype('float32').as_in_context(ctx),
                    end_label.astype('float32').as_in_context(ctx)]).mean()

                if accumulate:
                    ls = ls / accumulate
            ls.backward()
            # update
            if not accumulate or (batch_id + 1) % accumulate == 0:
                trainer.allreduce_grads()
                nlp.utils.clip_grad_global_norm(params, 1)
                trainer.update(1)

            step_loss += ls.asscalar()
        
#         for batch_id, data in enumerate(train_dataloader):
#             # set new lr
#             step_num = set_new_lr(step_num, batch_id)
#             # forward and backward
# #             with mx.autograd.record():
#             _, inputs, token_types, valid_length, start_label, end_label = data

#             log_num += len(inputs)
#             total_num += len(inputs)

#             def split_and_load(data, ctx):
#                 n, k = data.shape[0], len(ctx)
#                 print(n, k)
#                 if (n//k)*k != n:
#                     drop = n - (n//k)*k
#                     data = data[:-drop]
#                     n, k = data.shape[0], len(ctx)
#                 assert (n//k)*k == n, '# examples is not divided by # devices'
#                 idx = list(range(0, n+1, n//k))
#                 return [data[idx[i]:idx[i+1]].as_in_context(ctx[i]) for i in range(k)]
            
# #                 def train_batch(inputs, params, ctx, lr):
#                     # split the data batch and load them on GPUs
#             print(len(inputs[0]), len(token_types[0]), len(valid_length[0]), len(start_label[0]), len(end_label[0]))
#             inputs = split_and_load(inputs[0], ctx)
#             token_types = split_and_load(token_types[0], ctx)
# #             valid_length = split_and_load(valid_length, ctx)
# #             start_label = split_and_load(start_label[0], ctx)
# #             end_label = split_and_load(end_label[0], ctx)

#             # run forward on each GPU
#             with mx.autograd.record():
#                 losses = [loss_function(net(X, Y, W), [U, V])
#                           for X, Y, W, U, V in zip(inputs, token_types, valid_length, start_label, end_label)]
#             # run backward on each gpu
#             for ls in losses:
#                 ls.backward()
#                 step_loss += ls.asscalar()
#             # aggregate gradient over GPUs
#             for i in range(len(params[0])):
#                 allreduce([params[c][i].grad for c in range(len(ctx))])
#             # update parameters with SGD on each GPU
#             for p in params:
#                 nlp.utils.clip_grad_global_norm(P, 1)
#                 trainer.update(1)
            if (batch_id + 1) % log_interval == 0: 
                toc = time.time()
                log.info('Epoch: {}, Batch: {}/{}, Loss={:.4f}, lr={:.7f} Time cost={:.1f} Thoughput={:.2f} samples/s'  # pylint: disable=line-too-long
                         .format(epoch_id, batch_id, len(train_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, toc - tic, log_num/(toc - tic)))
                tic = time.time()
                step_loss = 0.0
                log_num = 0
        epoch_toc = time.time()
        log.info('Time cost={:.2f} s, Thoughput={:.2f} samples/s'.format(
            epoch_toc - epoch_tic, total_num/(epoch_toc - epoch_tic)))

    net.save_parameters(os.path.join(output_dir, 'net.params'))


train(log, train_data_transform, train_dataloader)

In [ ]:
def evaluate():
    """Evaluate the model on validation dataset.
    """
    log.info('Loading dev data...')
#     if version_2:
#         dev_data = SQuAD('dev', version='2.0')
#     else:
    dev_data = SQuAD('dev', version='1.1')
    if args.debug:
        sampled_data = [dev_data[0], dev_data[1], dev_data[2]]
        dev_data = mx.gluon.data.SimpleDataset(sampled_data)
    log.info('Number of records in dev data:{}'.format(len(dev_data)))

    dev_dataset = dev_data.transform(
        SQuADTransform(
            copy.copy(tokenizer),
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_pad=False,
            is_training=False)._transform, lazy=False)
    
    dev_data_transform, _ = preprocess_dataset(
        dev_data, SQuADTransform(
            copy.copy(tokenizer),
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_pad=False,
            is_training=False))
    log.info('The number of examples after preprocessing:{}'.format(
        len(dev_data_transform)))

    dev_dataloader = mx.gluon.data.DataLoader(
        dev_data_transform,
        batchify_fn=batchify_fn,
        num_workers=4, batch_size=test_batch_size,
        shuffle=False, last_batch='keep')

    log.info('start prediction')

    all_results = collections.defaultdict(list)

    epoch_tic = time.time()
    total_num = 0
    for data in dev_dataloader:
        example_ids, inputs, token_types, valid_length, _, _ = data
        total_num += len(inputs)
        out = net(inputs.astype('float32').as_in_context(ctx),
                  token_types.astype('float32').as_in_context(ctx),
                  valid_length.astype('float32').as_in_context(ctx))

        output = mx.nd.split(out, axis=2, num_outputs=2)
        example_ids = example_ids.asnumpy().tolist()
        pred_start = output[0].reshape((0, -3)).asnumpy()
        pred_end = output[1].reshape((0, -3)).asnumpy()

        for example_id, start, end in zip(example_ids, pred_start, pred_end):
            all_results[example_id].append(PredResult(start=start, end=end))
            
    epoch_toc = time.time()
    log.info('Time cost={:.2f} s, Thoughput={:.2f} samples/s'.format(
        epoch_toc - epoch_tic, total_num/(epoch_toc - epoch_tic)))

    log.info('Get prediction results...')

    all_predictions = collections.OrderedDict()

    for features in dev_dataset:
        results = all_results[features[0].example_id]
        example_qas_id = features[0].qas_id

        prediction, _ = predict(
            features=features,
            results=results,
            tokenizer=nlp.data.BERTBasicTokenizer(lower=lower),
            max_answer_length=max_answer_length,
            null_score_diff_threshold=null_score_diff_threshold,
            n_best_size=n_best_size,
#             version_2=version_2
        )

        all_predictions[example_qas_id] = prediction

    with io.open(os.path.join(output_dir, 'predictions.json'),
                 'w', encoding='utf-8') as fout:
        data = json.dumps(all_predictions, ensure_ascii=False)
        fout.write(data)

#     if version_2:
#         log.info('Please run evaluate-v2.0.py to get evaluation results for SQuAD 2.0')
#     else:
    F1_EM = get_F1_EM(dev_data, all_predictions)
    log.info(F1_EM)

In [ ]:
evaluate()

## Deploy on SageMaker

1. Saving the model parameters
2. Preparing functions for inference 
3. Building a docker container with dependencies installed
4. Launch a serving end-point with SageMaker SDK

### 1. Save the model parameters

In [17]:
## save parameters, model definition and vocabulary in a zip file

# net.export('checkpoint')

output_dir = "model_outputs"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

with open(output_dir+'/vocab.json', 'w') as f:
    f.write(vocab.to_json())
import tarfile
with tarfile.open(output_dir+"/model.tar.gz", "w:gz") as tar:
    tar.add("./temp/bert_qa-7eb11865.params")
#     tar.add("output_dir/checkpoint-0000.params") 
#     tar.add("output_dir/checkpoint-symbol.json") ???
    tar.add(output_dir+"/vocab.json")

### 2. Preparing functions for inference

Two functions: 
1. model_fn() to load model parameters
2. transform_fn() to run model inference given an input

### 3. Building a docker container with dependencies installed

Let's prepare a docker container with all the dependencies required for model inference. Here we build a docker container based on the SageMaker MXNet inference container, and you can find the list of all available inference containers at https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html

Here we use local mode for demonstration purpose. To deploy on actual instances, you need to login into AWS elastic container registry (ECR) service, and push the container to ECR. 

```
docker build -t $YOUR_EDR_DOCKER_TAG . -f Dockerfile
$(aws ecr get-login --no-include-email --region $YOUR_REGION)
docker push $YOUR_EDR_DOCKER_TAG
```

In [18]:
%%writefile Dockerfile

ARG REGION
FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/mxnet-inference:1.6.0-gpu-py3

RUN pip install --upgrade --user --pre 'mxnet-mkl' 'https://github.com/dmlc/gluon-nlp/tarball/v0.9.x'

RUN pip list | grep mxnet

COPY *.py /opt/ml/model/code/

Writing Dockerfile


Docker login cmd

In [19]:
!$(aws ecr get-login --no-include-email --region us-east-1 --registry-ids 763104351884)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [20]:
!export REGION=$(wget -qO- http://169.254.169.254/latest/meta-data/placement/availability-zone) &&\
 docker build --no-cache --build-arg REGION=${REGION::-1} -t my-docker:inference . -f Dockerfile

Sending build context to Docker daemon  843.8MB
Step 1/5 : ARG REGION
Step 2/5 : FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/mxnet-inference:1.6.0-gpu-py3
1.6.0-gpu-py3: Pulling from mxnet-inference

7927d38a: Pulling fs layer 
ac894db4: Pulling fs layer 
2af6d627: Pulling fs layer 
86211d23: Pulling fs layer 
603ff777: Pulling fs layer 
7165632f: Pulling fs layer 
96e40dcf: Pulling fs layer 
91ff3706: Pulling fs layer 
02a4385b: Pulling fs layer 
e229cfdb: Pulling fs layer 
0e6ed5b5: Pulling fs layer 
c8e328fe: Pulling fs layer 
bb20abb1: Pulling fs layer 
0702cb67: Pulling fs layer 
d6c2671b: Pulling fs layer 
486e676d: Pulling fs layer 
1ff3706: Waiting fs layer 
7d871d5a: Pulling fs layer 
c8e328fe: Waiting fs layer 
bb20abb1: Waiting fs layer 
c02fa024: Pulling fs layer 
1ff3706: Extracting  688.7MB/688.7MBBKfailed to register layer: Error processing tar file(exit status 1): write /usr/local/cuda-10.1/targets/x86_64-linux/lib/libcusolver.so.10.2.0.243: no space left on device


In [ ]:
!wget -qO- http://169.254.169.254/latest/meta-data/placement/availability-zone

In [ ]:
# !aws ecr get-login --no-include-email --registry-ids 763104351884
!aws ecr get-login --no-include-email --region us-east-1 --registry-ids 763104351884

In [ ]:
!docker pull 763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-gpu-py3
#              763104351884.dkr.ecr.<region>.amazonaws.com/mxnet-inference:1.4.1-gpu-py3

In [ ]:
!aws ecr get-login --no-include-email --region us-east-1

In [ ]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-gpu-py3


In [ ]:
import sagemaker

role=sagemaker.get_execution_role()


In [ ]:
!aws ecr get-login --no-include-email --registry-ids 520713654638